In [1]:
import pandas as pd
import numpy as np
from os import walk, makedirs

In [2]:
csvout_dir = "csvout/problem2/"
x_runs_dirs = next(walk(csvout_dir))[1]

In [8]:
outputs = {}
for dir in x_runs_dirs:
  files = next(walk(csvout_dir+dir))[2]
  curr_run = list()
  colnames = ['thread']
  for i in range(1,13):
    colnames.append("pass_"+str(i))
  colnames.append("total_time")
  curr_run.append(colnames)

  for file in files:
    col = pd.read_csv(f"{csvout_dir}{dir}/{file}").iloc[:, 1]
    times = col.values.tolist()
    append_list = [file]
    append_list.extend(times)
    curr_run.append(append_list)
    
  df = pd.DataFrame(np.row_stack(curr_run))
  ## add to the list ##
  outputs.update({dir : df})

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 14 and the array at index 1 has size 13